In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data.head()

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  

In [4]:
data.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
X,y = data.drop('target'),data['target']

In [6]:
X,y = data.drop('target',axis=1),data['target']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [9]:
import torch
import torch.nn as nn

In [10]:
import numpy as np

In [11]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [12]:
X_train.shape

torch.Size([227, 13])

In [13]:
X_test.shape

torch.Size([76, 13])

In [14]:
y_train.shape

torch.Size([227])

In [15]:
y_test.shape

torch.Size([76])

In [16]:
import torch.nn.functional as F

In [17]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,16)
        self.fc2 = nn.Linear(16,32)
        self.fc3 = nn.Linear(32,64)
        self.fc4 = nn.Linear(64,128)
        self.fc5 = nn.Linear(128,256)
        self.fc6 = nn.Linear(256,128)
        self.fc7 = nn.Linear(128,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [18]:
model = Test_Model()

In [19]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)

In [20]:
criterion = nn.L1Loss()

In [21]:
EPOCHS = 100

In [22]:
X_train.to(device)
y_train.to(device)
X_test.to(device)
y_test.to(device)

In [23]:
device = torch.device('cuda')

In [24]:
model = Test_Model().to(device)

In [25]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)

In [26]:
criterion = nn.L1Loss()

In [27]:
EPOCHS = 100

In [28]:
X_train.to(device)
y_train.to(device)
X_test.to(device)
y_test.to(device)

tensor([0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
        0., 0., 0., 0.], device='cuda:0')

In [29]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [30]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [31]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [32]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in range(EPOCHS):
    preds = model(X_train.float().to(device))
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [33]:
import wandb

In [34]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in range(EPOCHS):
    preds = model(X_train.float().to(device))
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()